In [81]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [82]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
train_features = pd.read_csv('/content/drive/My Drive/moa/train_features.csv')
train_targets_scored = pd.read_csv('/content/drive/My Drive/moa/train_targets_scored.csv')

In [85]:
train_features.dtypes.sort_values()

X = train_features.iloc[:, 1:].values
y = train_targets_scored.iloc[:, 1:].values

In [86]:
# oneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [87]:
X[:, 0:5]

array([[0.0, 1.0, 24, 'D1', 1.062],
       [0.0, 1.0, 72, 'D1', 0.0743],
       [0.0, 1.0, 48, 'D1', 0.628],
       ...,
       [1.0, 0.0, 48, 'D2', 0.3942],
       [0.0, 1.0, 24, 'D1', 0.666],
       [0.0, 1.0, 72, 'D1', -0.8598]], dtype=object)

In [88]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [89]:
X[:, 0:5]

array([[1.0, 0.0, 0.0, 1.0, 24],
       [1.0, 0.0, 0.0, 1.0, 72],
       [1.0, 0.0, 0.0, 1.0, 48],
       ...,
       [0.0, 1.0, 1.0, 0.0, 48],
       [1.0, 0.0, 0.0, 1.0, 24],
       [1.0, 0.0, 0.0, 1.0, 72]], dtype=object)

In [90]:
X.shape

(23814, 877)

In [91]:
X

array([[1.0, 0.0, 0.0, ..., 0.2139, 0.3801, 0.4176],
       [1.0, 0.0, 0.0, ..., 0.1241, 0.6077, 0.7371],
       [1.0, 0.0, 0.0, ..., -0.2187, -1.408, 0.6931],
       ...,
       [0.0, 1.0, 1.0, ..., 0.7592, 0.6656, 0.3808],
       [1.0, 0.0, 0.0, ..., 0.7015, -0.629, 0.074],
       [1.0, 0.0, 0.0, ..., -0.4775, -2.15, -4.252]], dtype=object)

In [92]:
# Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [93]:
X.shape

(23814, 877)

In [94]:
y.shape

(23814, 206)

In [95]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(877,1024)           
        self.fc2 = nn.Linear(1024,206)         
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

model = Net()
model

Net(
  (fc1): Linear(in_features=877, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=206, bias=True)
)

In [97]:
loss_fn = nn.BCEWithLogitsLoss()                       
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

batch_size = 64
n_epochs = 100
batch_no = len(X) // batch_size                 # batch_no=125, pois 125 * 64 = 8000

train_loss = 0
train_loss_min = np.Inf
for epoch in range(n_epochs):
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        x_var = Variable(torch.FloatTensor(X[start:end]))
        y_var = Variable(torch.FloatTensor(y[start:end])) 
        
        
        # output = model(x_var)
        # loss = loss_fn(output,y_var)

        optimizer.zero_grad()
        output = model(x_var)       
        loss = loss_fn(output,y_var)
        loss.backward()
        optimizer.step()
        
        # values, labels = torch.max(output, 1)
        # num_right = np.sum(labels.data.numpy() == y[start:end])
        train_loss += loss.item() * batch_size
    
    train_loss = train_loss / len(X)
    if train_loss <= train_loss_min:
        print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
        torch.save(model.state_dict(), "model.pt")
        train_loss_min = train_loss
    
    # if epoch % 200 == 0:
    #     print('')
    #     print("Epoch: {} \tTrain Loss: {} \tTrain Accuracy: {}".format(epoch + 1, train_loss))
    print("Ronaldo")

print('Training Ended! ')



Validation loss decreased (   inf ===> 0.695385). Saving the model...
Ronaldo
Ronaldo
Ronaldo
Ronaldo
Ronaldo


KeyboardInterrupt: ignored